In [123]:
# ! pip install transformers[torch] datasets evaluate seqeval -q

In [3]:
# !pip install ipywidgets

In [22]:
!pip install evaluate seqeval -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Load DarNER

In [5]:
import pandas as pd
df = pd.read_csv('DarNERcorp_train.csv')
df

,Unnamed: 0,Sentence,Token,Tag
0,0,0,Uppsala,B-LOC
1,1,0,),O
2,2,0,هيّا,O
3,3,0,رابع,O
4,4,0,أكبر,O
...,...,...,...,...
52719,52719,1984,خدّامين,O
52720,52720,1984,بحال,O
52721,52721,1984,لعبيد,O
52722,52722,1984,عند,O


In [6]:
df = df.dropna()
df.isna().sum()

Unnamed: 0    0
Sentence      0
Token         0
Tag           0
dtype: int64

In [7]:
label_list = sorted(list(df['Tag'].unique()), reverse=True)

In [8]:
label_list

['O', 'I-PER', 'I-ORG', 'I-MISC', 'I-LOC', 'B-PER', 'B-ORG', 'B-MISC', 'B-LOC']

In [9]:
id2label = {k:v for k,v in enumerate(label_list)}
label2id = {v:k for k,v in enumerate(label_list)}
label2id

{'O': 0,
 'I-PER': 1,
 'I-ORG': 2,
 'I-MISC': 3,
 'I-LOC': 4,
 'B-PER': 5,
 'B-ORG': 6,
 'B-MISC': 7,
 'B-LOC': 8}

In [10]:
token_lists = df.groupby('Sentence')['Token'].agg(list)
tag_lists = df.groupby('Sentence')['Tag'].agg(list)

In [11]:
data = []
for i,(tokens, tags) in enumerate(zip(token_lists, tag_lists)):
  r = {}
  r['id'] = i
  r['tokens'] = tokens
  r['ner_tags'] = [label2id[l] for l in tags]
  data.append(r)

In [12]:
dataset = pd.DataFrame(data)

In [13]:
dataset

,id,tokens,ner_tags
0,0,"[Uppsala, ), هيّا, رابع, أكبر, مدينة, ف, سّويد...","[8, 0, 0, 0, 0, 0, 0, 8, 0, 0, 8, 0, 8, 0, 8, 0]"
1,1,"[ف, 2018, كان, عدد, سّكان, ديالها, 172, ،, 402...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,2,"[أوپيك, (, ب, لينݣليزية, :, OPEC, -, تاتعني, ل...","[6, 0, 0, 7, 0, 6, 0, 0, 6, 2, 2, 2, 2, 2, 0, ..."
3,3,"[تأسسات, هاد, لمنضمة, نهار, 14, شتنبر, 1960, ف...","[0, 0, 0, 0, 7, 3, 3, 0, 8, 0, 0, 0, 0, 0, 0, ..."
4,4,"[ف, 1965, ،, رحلات, لمنضمة, ،, و, ولأّ, لمقر, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 8, ..."
...,...,...,...
1980,1980,"[ف, لغرب, شدّات, ؤكرانيا, لي, كانت, محتلاها, پ...","[0, 0, 0, 8, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, ..."
1981,1981,"[دخول, ؤكرانيا, تحت, سّلطة, د, موسكوڤيا, دخّل,...","[0, 8, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1982,1982,"[تّسارات, كانو, سيرتو, مهتمين, ب, طّيكنولوجيا,...","[0, 0, 0, 0, 0, 0, 0, 0]"
1983,1983,"[ف, 1721, ،, تّسار, پيوطر, لعاضيم, زاد, ل, راس...","[0, 0, 0, 0, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
from datasets import load_dataset, Dataset
dataset = Dataset.from_pandas(dataset)

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT-arabizi")

tokenizer_config.json: 100%|███████████████████████████████████████████████████████████| 347/347 [00:00<00:00, 1.51MB/s]
vocab.txt: 100%|█████████████████████████████████████████████████████████████████████| 787k/787k [00:00<00:00, 3.19MB/s]
tokenizer.json: 100%|██████████████████████████████████████████████████████████████| 1.67M/1.67M [00:00<00:00, 4.06MB/s]
special_tokens_map.json: 100%|██████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 794kB/s]


In [16]:
dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 1985
})

In [17]:
example = dataset[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'up',
 '##ps',
 '##ala',
 ')',
 'هيا',
 'رابع',
 'اكبر',
 'مدينة',
 'ف',
 'سوي',
 '##د',
 'من',
 'بعد',
 'سط',
 '##وك',
 '##هول',
 '##م',
 '،',
 'ݣ',
 '##وت',
 '##نب',
 '##ور',
 '##ݣ',
 'و',
 'مال',
 '##مو',
 '.',
 '[SEP]']

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████| 1985/1985 [00:00<00:00, 5413.44 examples/s]


In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

2024-01-13 21:00:14.050275: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 21:00:14.050380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 21:00:14.191792: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-13 21:00:14.495516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Evaluate

In [23]:
import evaluate

seqeval = evaluate.load("seqeval")

In [24]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Train

In [25]:
id2label, len(id2label)

({0: 'O',
  1: 'I-PER',
  2: 'I-ORG',
  3: 'I-MISC',
  4: 'I-LOC',
  5: 'B-PER',
  6: 'B-ORG',
  7: 'B-MISC',
  8: 'B-LOC'},
 9)

In [26]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "SI2M-Lab/DarijaBERT-arabizi", num_labels=len(id2label), id2label=id2label, label2id=label2id
)

config.json: 100%|█████████████████████████████████████████████████████████████████████| 787/787 [00:00<00:00, 5.01MB/s]
model.safetensors: 100%|███████████████████████████████████████████████████████████| 1.02G/1.02G [00:27<00:00, 36.7MB/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT-arabizi and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
training_args = TrainingArguments(
    output_dir="ner-DarijaBERT-arabizi",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/envs/ml/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.324015,0.539661,0.325125,0.405782,0.899630
2,No log,0.254093,0.559347,0.472037,0.511996,0.920345
3,No log,0.206221,0.569651,0.582846,0.576173,0.935009
4,No log,0.179141,0.616215,0.631260,0.623647,0.942597
5,No log,0.152811,0.650439,0.680301,0.665035,0.950868
6,No log,0.130773,0.688019,0.726210,0.706599,0.958247
7,No log,0.118864,0.712620,0.727045,0.719760,0.961206
8,No log,0.110036,0.730693,0.766068,0.747963,0.965114
9,No log,0.103672,0.742272,0.756678,0.749406,0.966670
10,No log,0.101141,0.747485,0.775250,0.761115,0.968149


Checkpoint destination directory ner-DarijaBERT-arabizi/checkpoint-32 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/envs/ml/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory ner-DarijaBERT-arabizi/checkpoint-64 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/envs/ml/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/ml/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along d

TrainOutput(global_step=320, training_loss=0.2234816312789917, metrics={'train_runtime': 421.1285, 'train_samples_per_second': 47.135, 'train_steps_per_second': 0.76, 'total_flos': 1968511827211272.0, 'train_loss': 0.2234816312789917, 'epoch': 10.0})

In [29]:
trainer.push_to_hub()

'https://huggingface.co/Oelbourki/ner-DarijaBERT-arabizi/tree/main/'

## Inference

In [30]:
texts = ["دونالد طرامب هو الرئيس لفايت د ميريكان",
         "لمقار ديال OPEC كاين ف فيينا العاصمة ديال لوتريش",
         "عوينة يغومان جماعة ترابية قروية كاينة ف إقليم آسا الزاݣ"]

In [47]:
from transformers import pipeline

classifier = pipeline("ner", model="Oelbourki/ner-DarijaBERT-arabizi",  aggregation_strategy="first")
token_classification = classifier(texts)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [67]:
for text, pred in zip(texts, token_classification):
    print(text)
    print(pred)
    print()

دونالد طرامب هو الرئيس لفايت د ميريكان
[{'entity_group': 'LOC', 'score': 0.9000089, 'word': 'ميريكان', 'start': 31, 'end': 38}]

لمقار ديال OPEC كاين ف فيينا العاصمة ديال لوتريش
[{'entity_group': 'MISC', 'score': 0.30474782, 'word': 'opec', 'start': 11, 'end': 15}, {'entity_group': 'LOC', 'score': 0.87563026, 'word': 'فيينا', 'start': 23, 'end': 28}, {'entity_group': 'LOC', 'score': 0.8966449, 'word': 'لوتريش', 'start': 42, 'end': 48}]

عوينة يغومان جماعة ترابية قروية كاينة ف إقليم آسا الزاݣ
[{'entity_group': 'LOC', 'score': 0.91604114, 'word': 'عوينة', 'start': 0, 'end': 5}, {'entity_group': 'LOC', 'score': 0.4215859, 'word': 'يغومان', 'start': 6, 'end': 12}, {'entity_group': 'LOC', 'score': 0.6283031, 'word': 'اسا', 'start': 46, 'end': 49}, {'entity_group': 'LOC', 'score': 0.72588617, 'word': 'الزاݣ', 'start': 50, 'end': 55}]

